In [9]:
from common_functions import get_secret, ret_metabase, google_sheets, dwh_query
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO

In [45]:
data = ret_metabase("Egypt", 63605)
data

/home/ec2-user/service_account_key.json


,RETAILER_ID,Workflow ID,Date,Description,AGENT_ID,COUPON_VALUE
0,573568,7,2025-10-05,تسجيل بينات العميل,772,NaN
1,573568,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,772,NaN
2,573568,5,2025-10-05,عميل غير نشط على سوق التجار اخر 60 يوم؛ كوبون ...,772,NaN
3,608839,7,2025-10-05,تسجيل بينات العميل,772,NaN
4,608839,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,772,NaN
...,...,...,...,...,...,...
9673,7719,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3404,20.0
9674,8327,1,2025-10-05,عميل نشط - مطلوب اوردر من بضاعة مكسب بقيمة194...,3404,NaN
9675,8327,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,3404,NaN
9676,8327,6,2025-10-05,تفعيل الآجل، مصاريف إدارية 1.75% في الاسبوع ، ...,3404,NaN


In [46]:
data = data[data["COUPON_VALUE"].notnull()]

data = data.sort_values(by="COUPON_VALUE", ascending=True)
data

,RETAILER_ID,Workflow ID,Date,Description,AGENT_ID,COUPON_VALUE
5564,826762,5,2025-10-05,عميل غير نشط على سوق التجار اكتر من 3 شهور؛ كو...,10750,20.0
6479,45426,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,2866,20.0
6580,90376,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3141,20.0
6574,52047,5,2025-10-05,عميل غير نشط على سوق التجار اخر 60 يوم؛ كوبون ...,3141,20.0
6567,23688,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3141,20.0
...,...,...,...,...,...,...
1118,533932,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,116,100.0
1113,496018,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,116,100.0
9489,248431,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10691,100.0
9491,248742,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10691,100.0


In [47]:
coupons  = data[["RETAILER_ID", "COUPON_VALUE"]]
coupons = data[["RETAILER_ID", "COUPON_VALUE"]].rename(
    columns={
        "RETAILER_ID": "retailer_id",
        "COUPON_VALUE": "amount"
    }
)

coupons["reason_id"] = 69
coupons["type_id"] = 4
coupons["redemption_method"] =''
coupons

,retailer_id,amount,reason_id,type_id,redemption_method
5564,826762,20.0,69,4,
6479,45426,20.0,69,4,
6580,90376,20.0,69,4,
6574,52047,20.0,69,4,
6567,23688,20.0,69,4,
...,...,...,...,...,...
1118,533932,100.0,69,4,
1113,496018,100.0,69,4,
9489,248431,100.0,69,4,
9491,248742,100.0,69,4,


In [9]:
coupons.to_excel("coupons.xlsx", index=False)


In [14]:
import pandas as pd

# Create the DataFrame
df = pd.DataFrame({
    'retailer_id': [361790],
    'amount': [20.0],
    'reason_id': [69],
    'type_id': [4]
    
})
df["redemption_method"] =''
df.to_excel("df.xlsx", index=False)


In [43]:
import requests
JT = get_jwt()

url = "https://api.maxab.info/commerce/api/admins/v1/wallet/transactions"

# Prepare file and form data
files = {
    "file": ("test.xlsx", open("df.xlsx", "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
}
data = {
    "compensation_coupon_rule_id": "343",
    "incentive_coupon_rule_id": "344"
}

headers = {
    "Authorization": f"Bearer {JT}"  # <-- replace with your actual token
}

# Perform the POST request
response = requests.post(url, headers=headers, files=files, data=data)

print(response.status_code)
print(response.text)


200
PK ��E[               [Content_Types].xml�S�n�0����*6�PU�C���\{�X�%����]8�R�
q�cfgfW�d�q�ZCB|��|�*�*h㻆},^�{Va�^K<4�6�NXQ�ǆ�9�!P��$��҆�d�c�D�j);��ѝP�g��E�M'O�ʕ����H7L�h���R���G��^�'�{�zސʮB��3�˙��h.�h�W�жF�j娄CQՠ똈���}ιL�U:D�����%އ����,�B���[�	�� ;˱�	�{N��~��X�p�ykOL��kN�V��ܿBZ~����q�� �ar��{O�PKz��q;    PK ��E[               _rels/.rels���j�0�_���8�`�Q��2�m��4[ILb��ږ���.[K
�($}��v?�I�Q.���uӂ�h���x>=��@��p�H"�~�}�	�n����*"�H�׺؁�����8�Z�^'�#��7m{��O�3���G�u�ܓ�'��y|a�����D�	��l_EYȾ� ���vql3�ML�eh���*���\3�Y0���oJ׏�	:����}PK��z��   I  PK ��E[               docProps/app.xmlM��
�0D�~EȽ��ADҔ��A? ��6�lB�J?ߜ���0���ͯ�)�@��׍H6���V>��$;�SC
�GS�b����l�&�e��L!y�%��49��`_���4G���F��J��Wg
6o�Z:.����| ��1�K��>F�		�-B��뵈��;�|���3�!ops/core.xmlm��J�0E%�M�R��ve@Pp�����bs!�v�{�:VPߒ�u'�����������bFZ5�����mv�Q��(1Z
%� �0s���J�J���E�$�4���a#x�X��<�a��iʧr��F��n����_������҃��P���R#��Syu��ⶠE�1��j�JN

In [8]:
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",    
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token


In [6]:
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]

In [ ]:
# fintech_service_account_emailname
# 

In [11]:
get_jwt()

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4cnE5b1ZqaXRFdjdNeExyTWpzS3NkdUl3eWdxZnZlVkM4T0Vfb3dvY3N3In0.eyJleHAiOjE3NTk0MTMyODEsImlhdCI6MTc1OTQxMjk4MSwianRpIjoiZWNjYWVlNTktOGE0Mi00Y2QzLWE4MGYtZmEzN2RkNjJiMmYwIiwiaXNzIjoiaHR0cHM6Ly9zc28ubWF4YWIuaW5mby9hdXRoL3JlYWxtcy9tYXhhYiIsImF1ZCI6WyJ3b3JrZm9yY2UiLCJlLW1vbmV5Iiwib3JkZXItbWFuYWdlbWVudCIsImNyZWRpdCIsIm1heGFiLWNvbW1lcmNlIiwiYWNjb3VudCJdLCJzdWIiOiJkYzVkZjcwMS1mODdkLTQwODUtODdmYy01YzI0ZWYzYTRiOGQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJhZG1pbi1wb3J0YWwiLCJzZXNzaW9uX3N0YXRlIjoiNjgxZjA5M2MtZjQ4OS00MmRlLWJkYjEtYmVkOGQ2NWI0YjNhIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHBzOi8vYWRtaW4ubWF4YWIuaW5mbyJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1tYXhhYiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJ3b3JrZm9yY2UiOnsicm9sZXMiOlsib3JkZXItbWFuYWdtZW50LWFwaXMtaW50ZWdyYXRpb24iLCJ3b3JrZm9yY2UtaW50ZWdyYXRpb24iXX0sImUtbW9uZXkiOnsicm9sZXMiOlsiZS1tb25leSJdfSwib3JkZXItbWFuYWdlbWVudCI6eyJyb2x

In [2]:
data = ret_metabase("egypt", 64814)
data

/home/ec2-user/service_account_key.json


,RETAILER_ID,COUPON_VALUE
0,79726,50
1,200140,50
2,905896,50
3,80048,50
4,84968,50
...,...,...
116,903183,50
117,742612,50
118,738621,50
119,764719,50


In [6]:
data = data.head(3)
data

,RETAILER_ID,COUPON_VALUE
0,79726,50
1,200140,50
2,905896,50


In [7]:
coupons = data[["RETAILER_ID", "COUPON_VALUE"]].rename(
columns={
    "RETAILER_ID": "retailer_id",
    "COUPON_VALUE": "amount"})

coupons["reason_id"] = 69
coupons["type_id"] = 4
coupons["redemption_method"] =''
coupons

,retailer_id,amount,reason_id,type_id,redemption_method
0,79726,50,69,4,
1,200140,50,69,4,
2,905896,50,69,4,


In [10]:
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",    
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token


data = ret_metabase("egypt", 64814)
coupons = data[["RETAILER_ID", "COUPON_VALUE"]].rename(
columns={
    "RETAILER_ID": "retailer_id",
    "COUPON_VALUE": "amount"})

coupons["reason_id"] = 69
coupons["type_id"] = 4
coupons["redemption_method"] =''

from io import BytesIO
import requests
import time
JT = get_jwt()

url = "https://api.maxab.info/commerce/api/admins/v1/wallet/transactions"
data = {
    "compensation_coupon_rule_id": "343",
    "incentive_coupon_rule_id": "344"
}
headers = {
    "Authorization": f"Bearer {JT}"
}

BATCH_SIZE = 500
total_rows = len(coupons)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
print(f"Uploading coupons in {total_batches} batch(es) of up to {BATCH_SIZE} rows...")

# Coupon dispatch tracking
successful_coupon_batches = 0
failed_coupon_batches = 0
total_coupon_rows = total_rows
total_coupon_amount = float(coupons['amount'].sum()) if total_rows > 0 else 0.0

for batch_index in range(total_batches):
    start_idx = batch_index * BATCH_SIZE
    end_idx = min((batch_index + 1) * BATCH_SIZE, total_rows)
    batch_df = coupons.iloc[start_idx:end_idx].copy()

    buf = BytesIO()
    with pd.ExcelWriter(buf, engine="xlsxwriter") as writer:
        batch_df.to_excel(writer, index=False)
    buf.seek(0)

    files = {
        "file": (f"coupons_batch_{batch_index + 1}_of_{total_batches}.xlsx", buf, "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    }

    print(f"Sending batch {batch_index + 1}/{total_batches} (rows {start_idx + 1}-{end_idx})")

    # Perform the POST request with simple 429 retry (sleep 3 minutes)
    max_429_retries = 4
    response = None
    for attempt in range(1, max_429_retries + 1):
        try:
            if hasattr(files.get("file", (None, None))[1], 'seek'):
                files["file"][1].seek(0)
        except Exception:
            pass

        response = requests.post(url, headers=headers, files=files, data=data)
        if response.status_code == 429 and attempt < max_429_retries:
            print(f"HTTP 429 received for batch {batch_index + 1} (attempt {attempt}/{max_429_retries}). Sleeping 180 seconds before retry...")
            time.sleep(180)
            continue
        break

    print(f"Batch {batch_index + 1}/{total_batches} response: {response.status_code}")
    try:
        print(response.text)
    except Exception:
        pass

    # Track success/failure per coupon batch
    if response is not None and response.status_code in [200, 201]:
        successful_coupon_batches += 1
    else:
        failed_coupon_batches += 1

Uploading coupons in 1 batch(es) of up to 500 rows...
Sending batch 1/1 (rows 1-3)
Batch 1/1 response: 200
PK cM[               [Content_Types].xml�S�n�0����*6�PU�C���\{�X�%����]8�R�
q�cfgfW�d�q�ZCB|��|�*�*h㻆},^�{Va�^K<4�6�NXQ�ǆ�9�!P��$��҆�d�c�D�j);��ѝP�g��E�M'O�ʕ����H7L�h���R���G��^�'�{�zސʮB��3�˙��h.�h�W�жF�j娄CQՠ똈���}ιL�U:D�����%އ����,�B���[�	�� ;˱�	�{N��~��X�p�ykOL��kN�V��ܿBZ~����q�� �ar��{O�PKz��q;    PK cM[               _rels/.rels���j�0�_���8�`�Q��2�m��4[ILb��ږ���.[K
�($}��v?�I�Q.���uӂ�h���x>=��@��p�H"�~�}�	�n����*"�H�׺؁�����8�Z�^'�#��7m{��O�3���G�u�ܓ�'��y|a�����D�	��l_EYȾ� ���vql3�ML�eh���*���\3�Y0���oJ׏�	:����}PK��z��   I  PK cM[               docProps/app.xmlM��
�0D�~EȽ��ADҔ��A? ��6�lB�J?ߜ���0���ͯ�)�@��׍H6���V>��$;�SC
�GS�b����l�&�e��L!y�%��49��`_���4G���F��J��Wg
~�PK�|wؑ   �   PK cM[               docProps/core.xmlm�]K�0��J�}��uCC�!�@P8�x�c[l>H���޴�
�]��9'o�9����55f9���7m�